In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import datetime


In [2]:
#Import data, check its worked
stock_data = pd.read_csv('Stockreturn.csv')
return_data = pd.read_csv('OutstandingShares.csv')
mapping_data = pd.read_csv('Companyprofiles.csv')
stock_data.head(5)
# return_data.head(1)

,Unnamed: 0,Adj Close**,Close,High,Low,Open,Ticker,time
0,0,24.13,25.70,26.48,20.53,20.92,AALB.AS,20200401
1,1,37.57,40.01,40.92,37.08,38.59,AALB.AS,20191201
2,2,31.69,33.75,43.11,32.96,39.57,AALB.AS,20200201
3,3,37.11,39.52,41.06,39.13,40.01,AALB.AS,20200101
4,4,28.37,29.19,30.89,24.75,25.00,AALB.AS,20200601


In [3]:
#Double check the data type
type(stock_data)
stock_data.dtypes

Unnamed: 0       int64
Adj Close**    float64
Close          float64
High           float64
Low            float64
Open           float64
Ticker          object
time             int64
dtype: object

In [4]:
#Drop unnecessary data
columns = ['Ticker', 'time', 'Open', 'High', 'Low','Close']
stock_data = stock_data[columns]
stock_data.head(10)
#convert time to datetime


,Ticker,time,Open,High,Low,Close
0,AALB.AS,20200401,20.92,26.48,20.53,25.70
1,AALB.AS,20191201,38.59,40.92,37.08,40.01
2,AALB.AS,20200201,39.57,43.11,32.96,33.75
3,AALB.AS,20200101,40.01,41.06,39.13,39.52
4,AALB.AS,20200601,25.00,30.89,24.75,29.19
5,AALB.AS,20200301,34.51,34.68,16.16,21.64
6,AALB.AS,20200501,24.74,25.60,21.15,24.65
7,AALB.AS,20191101,36.15,39.80,36.01,38.45
8,AALB.AS,20191001,36.40,37.96,34.05,36.06
9,ABN.AS,20200401,7.24,8.30,6.23,7.00


In [6]:
#Left merge tables on ticker
new_table = stock_data.merge(right = return_data, on='Ticker', how='left')
new_table.head(5)

#Left merge tables on ticker to add industry mappings
new_table = mapping_data.merge(right = new_table, on='Ticker', how='left')
new_table.head(5)
new_table.time = new_table.time.fillna(0).astype(int)

In [8]:
new_table = new_table[new_table['time'] !=0]

In [9]:
new_table.time = new_table.time.fillna(0).astype(int).apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d')  )

In [10]:
#Change all column names to lower case
new_table.columns= new_table.columns.str.lower()
new_table.head(5)

,company,full time employees,industry,sector(s),ticker,time,open,high,low,close,unnamed: 0,index,basic average shares
0,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-04-01,20.92,26.48,20.53,25.70,2.00,"2,019.00","110,580.00"
1,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2019-12-01,38.59,40.92,37.08,40.01,2.00,"2,019.00","110,580.00"
2,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-02-01,39.57,43.11,32.96,33.75,2.00,"2,019.00","110,580.00"
3,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-01-01,40.01,41.06,39.13,39.52,2.00,"2,019.00","110,580.00"
4,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-06-01,25.00,30.89,24.75,29.19,2.00,"2,019.00","110,580.00"


In [11]:
#Calculate market value
new_table['market value'] = new_table['close'] * new_table['basic average shares']
new_table['market value low'] = new_table['low'] * new_table['basic average shares']
new_table.head(5)


,company,full time employees,industry,sector(s),ticker,time,open,high,low,close,unnamed: 0,index,basic average shares,market value,market value low
0,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-04-01,20.92,26.48,20.53,25.70,2.00,"2,019.00","110,580.00","2,841,906.00","2,270,207.40"
1,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2019-12-01,38.59,40.92,37.08,40.01,2.00,"2,019.00","110,580.00","4,424,305.80","4,100,306.40"
2,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-02-01,39.57,43.11,32.96,33.75,2.00,"2,019.00","110,580.00","3,732,075.00","3,644,716.80"
3,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-01-01,40.01,41.06,39.13,39.52,2.00,"2,019.00","110,580.00","4,370,121.60","4,326,995.40"
4,Aalberts N.V.,"16,094",Specialty Industrial Machinery,Industrials,AALB.AS,2020-06-01,25.00,30.89,24.75,29.19,2.00,"2,019.00","110,580.00","3,227,830.20","2,736,855.00"


In [12]:
df = new_table.groupby(['time'])['market value','market value low'].sum()
df['market value low'].diff()

<ipython-input-12-7f11d0b38b7f>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = new_table.groupby(['time'])['market value','market value low'].sum()


time
2019-10-01               nan
2019-11-01     36,406,867.29
2019-12-01     -5,609,015.82
2020-01-01    -12,484,952.08
2020-02-01   -131,846,523.61
2020-03-01   -385,011,569.02
2020-04-01    171,945,477.16
2020-05-01     39,145,499.58
2020-06-01     84,877,541.31
Name: market value low, dtype: float64

In [66]:
# summary_columns = ['ticker', 'time', 'close', 'total revenue', 'gross profit', 'market value']
# summary_table = new_table[summary_columns]
# summary_table.head(5)

,ticker,time,close,total revenue,gross profit,market value
0,AALB.AS,20200401,25.70,NaN,NaN,NaN
1,AALB.AS,20191201,40.01,NaN,NaN,NaN
2,AALB.AS,20200201,33.75,NaN,NaN,NaN
3,AALB.AS,20200101,39.52,NaN,NaN,NaN
4,AALB.AS,20200601,29.19,NaN,NaN,NaN


In [77]:
# #Group by tickers and time
# sorted_summary_table = summary_table.sort_values(by =['ticker', 'time'], ascending = True )
# sorted_summary_table.head(15)

,ticker,time,close,total revenue,gross profit,market value
170,AAA.AS,20191001,0.12,NaN,NaN,NaN
169,AAA.AS,20191101,0.10,NaN,NaN,NaN
163,AAA.AS,20191201,0.10,NaN,NaN,NaN
165,AAA.AS,20200101,0.11,NaN,NaN,NaN
164,AAA.AS,20200201,0.12,NaN,NaN,NaN
167,AAA.AS,20200301,0.08,NaN,NaN,NaN
162,AAA.AS,20200401,0.07,NaN,NaN,NaN
168,AAA.AS,20200501,0.06,NaN,NaN,NaN
166,AAA.AS,20200601,0.08,NaN,NaN,NaN
8,AALB.AS,20191001,36.06,NaN,NaN,NaN
